# Training GRU model

In [1]:
import os

import keras
import numpy as np
import pandas as pd

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
from utils.KerasModels import generateExpData
from utils.KerasModels import buildCharGRUModel
from sklearn.metrics import balanced_accuracy_score, f1_score, precision_score, recall_score

MODELS_DIR = os.path.join('..', 'models', 'charCNN_GRU')
EPOCHS = 200

In [2]:
def prepare_data(data_directory):
    train = pd.read_csv(os.path.join(data_directory, 'train.csv'))
    validation = pd.read_csv(os.path.join(data_directory, 'validation.csv'))
    test = pd.read_csv(os.path.join(data_directory, 'test.csv'))

    dfTrainDataset = train[["screen_name", "text", "account.type"]]
    dfValDataset = validation[["screen_name", "text", "account.type"]]
    dfTestDataset = test[["screen_name", "text", "account.type"]]

    tokenizer = None
    train_features, tokenizer = generateExpData(dfTrainDataset, tokenizer=tokenizer)
    val_features, tokenizer = generateExpData(dfValDataset, tokenizer=tokenizer)
    test_features, tokenizer = generateExpData(dfTestDataset, tokenizer=tokenizer)

    dictLabels = {"human": 0, "bot": 1}
    y_train = dfTrainDataset["account.type"].apply(lambda x: dictLabels[x])
    y_val = dfValDataset["account.type"].apply(lambda x: dictLabels[x])
    y_test = dfTestDataset["account.type"].apply(lambda x: dictLabels[x])

    train_labels = y_train.tolist()
    val_labels = y_val.tolist()
    test_labels = y_test.tolist()

    vocab_size = len(tokenizer.word_index)

    return train_features, val_features, test_features, train_labels, val_labels, test_labels, vocab_size


def proba_to_pred(y_proba):
    y_pred_char_cnn = (y_proba > 0.5).astype(int)
    return y_pred_char_cnn


def calculate_metrics(y_true, y_pred):
    results = {
        'balanced_accuracy': balanced_accuracy_score(y_true, y_pred),
        'f1_score': f1_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred),
        'recall': recall_score(y_true, y_pred)
    }
    return results

In [3]:
DATA_PATH = os.path.join('..', 'data', 'preprocessed_url_simple')
train_features, val_features, test_features, train_labels, val_labels, test_labels, vocab_size = prepare_data(DATA_PATH)

model = buildCharGRUModel(vocab_size, embSize=32, inputSize=320)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 320)]             0         
                                                                 
 embedding (Embedding)       (None, 320, 32)           3360      
                                                                 
 bidirectional (Bidirectiona  (None, 1024)             1677312   
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense (Dense)               (None, 1)                 1025      
                                                                 
Total params: 1,681,697
Trainable params: 1,681,697
Non-trainable params: 0
___________________________________________________

In [4]:
# Train CNN model.
os.makedirs(MODELS_DIR, exist_ok=True)
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
save_best = keras.callbacks.ModelCheckpoint(os.path.join(MODELS_DIR, "simple"), monitor='val_loss',
                                            save_best_only=True, restore_best_weights=True)
model.fit(np.array(train_features), np.array(train_labels), batch_size=256, epochs=EPOCHS,
          validation_data=(np.array(val_features), np.array(val_labels)),
          callbacks=[early_stopping, save_best])

Epoch 1/200
81/81 [==============================] - ETA: 0s - loss: 0.6135 - accuracy: 0.6275

INFO:tensorflow:Assets written to: ..\models\charCNN_GRU\simple\assets


INFO:tensorflow:Assets written to: ..\models\charCNN_GRU\simple\assets


81/81 [==============================] - 14s 140ms/step - loss: 0.6135 - accuracy: 0.6275 - val_loss: 0.5764 - val_accuracy: 0.6573
Epoch 2/200
81/81 [==============================] - 7s 83ms/step - loss: 0.6181 - accuracy: 0.6408 - val_loss: 0.5846 - val_accuracy: 0.6364
Epoch 3/200
81/81 [==============================] - ETA: 0s - loss: 0.5781 - accuracy: 0.6575

INFO:tensorflow:Assets written to: ..\models\charCNN_GRU\simple\assets


...

INFO:tensorflow:Assets written to: ..\models\charCNN_GRU\simple\assets


81/81 [==============================] - 11s 136ms/step - loss: 0.3429 - accuracy: 0.8427 - val_loss: 0.3703 - val_accuracy: 0.8245
Epoch 34/200
81/81 [==============================] - 7s 83ms/step - loss: 0.3466 - accuracy: 0.8402 - val_loss: 0.3722 - val_accuracy: 0.8258
Epoch 35/200
81/81 [==============================] - ETA: 0s - loss: 0.3379 - accuracy: 0.8454

INFO:tensorflow:Assets written to: ..\models\charCNN_GRU\simple\assets


INFO:tensorflow:Assets written to: ..\models\charCNN_GRU\simple\assets


81/81 [==============================] - 11s 136ms/step - loss: 0.3379 - accuracy: 0.8454 - val_loss: 0.3670 - val_accuracy: 0.8236
Epoch 36/200
81/81 [==============================] - 7s 84ms/step - loss: 0.6344 - accuracy: 0.7026 - val_loss: 0.6887 - val_accuracy: 0.5426
Epoch 37/200
81/81 [==============================] - 7s 83ms/step - loss: 0.6923 - accuracy: 0.5684 - val_loss: 0.6324 - val_accuracy: 0.6064
Epoch 38/200
81/81 [==============================] - 7s 84ms/step - loss: 0.6498 - accuracy: 0.6067 - val_loss: 0.6157 - val_accuracy: 0.6177
Epoch 39/200
81/81 [==============================] - 7s 82ms/step - loss: 0.6303 - accuracy: 0.6187 - val_loss: 0.6150 - val_accuracy: 0.6138
Epoch 40/200
81/81 [==============================] - 7s 82ms/step - loss: 0.6221 - accuracy: 0.6247 - val_loss: 0.6043 - val_accuracy: 0.6182
Epoch 41/200
81/81 [==============================] - 7s 84ms/step - loss: 0.6149 - accuracy: 0.6253 - val_loss: 0.6000 - val_accuracy: 0.6286
Epoch 42/2

In [5]:
probas = model.predict(np.array(test_features))
y_pred = proba_to_pred(probas)
results = calculate_metrics(test_labels, y_pred)
results

80/80 [==============================] - 1s 15ms/step


{'balanced_accuracy': 0.8322305359937403,
 'f1_score': 0.8443960826985855,
 'precision': 0.7880839539607312,
 'recall': 0.909375}

# Lemmatized data 

In [6]:
LEMMATIZED_DATA_PATH = os.path.join('..', 'data', 'lemmatized')
lem_train_features, lem_val_features, lem_test_features, lem_train_labels, lem_val_labels, lem_test_labels, lem_vocab_size = prepare_data(
    LEMMATIZED_DATA_PATH)

modelLEM = buildCharGRUModel(lem_vocab_size, embSize=32, inputSize=320)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 320)]             0         
                                                                 
 embedding_1 (Embedding)     (None, 320, 32)           3360      
                                                                 
 bidirectional_1 (Bidirectio  (None, 1024)             1677312   
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 1024)              0         
                                                                 
 dense_1 (Dense)             (None, 1)                 1025      
                                                                 
Total params: 1,681,697
Trainable params: 1,681,697
Non-trainable params: 0
_________________________________________________

In [7]:
# Train CNN model.
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
save_best = keras.callbacks.ModelCheckpoint(os.path.join(MODELS_DIR, "lemmatized"), monitor='val_loss',
                                            save_best_only=True, restore_best_weights=True)
modelLEM.fit(np.array(lem_train_features), np.array(lem_train_labels), batch_size=256, epochs=EPOCHS,
             validation_data=(np.array(lem_val_features), np.array(lem_val_labels)),
             callbacks=[early_stopping, save_best])

Epoch 1/200
81/81 [==============================] - ETA: 0s - loss: 0.6511 - accuracy: 0.6133

INFO:tensorflow:Assets written to: ..\models\charCNN_GRU\lemmatized\assets


INFO:tensorflow:Assets written to: ..\models\charCNN_GRU\lemmatized\assets


81/81 [==============================] - 13s 141ms/step - loss: 0.6511 - accuracy: 0.6133 - val_loss: 0.6047 - val_accuracy: 0.6186
Epoch 2/200
81/81 [==============================] - ETA: 0s - loss: 0.5934 - accuracy: 0.6328

INFO:tensorflow:Assets written to: ..\models\charCNN_GRU\lemmatized\assets


...

INFO:tensorflow:Assets written to: ..\models\charCNN_GRU\lemmatized\assets


81/81 [==============================] - 11s 132ms/step - loss: 0.3068 - accuracy: 0.8573 - val_loss: 0.3434 - val_accuracy: 0.8397
Epoch 36/200
81/81 [==============================] - ETA: 0s - loss: 0.3042 - accuracy: 0.8577

INFO:tensorflow:Assets written to: ..\models\charCNN_GRU\lemmatized\assets


INFO:tensorflow:Assets written to: ..\models\charCNN_GRU\lemmatized\assets


81/81 [==============================] - 11s 135ms/step - loss: 0.3042 - accuracy: 0.8577 - val_loss: 0.3423 - val_accuracy: 0.8423
Epoch 37/200
81/81 [==============================] - 7s 81ms/step - loss: 0.2986 - accuracy: 0.8610 - val_loss: 0.3482 - val_accuracy: 0.8401
Epoch 38/200
81/81 [==============================] - 7s 81ms/step - loss: 0.2963 - accuracy: 0.8600 - val_loss: 0.3480 - val_accuracy: 0.8427
Epoch 39/200
81/81 [==============================] - 7s 82ms/step - loss: 0.2938 - accuracy: 0.8625 - val_loss: 0.3549 - val_accuracy: 0.8445
Epoch 40/200
81/81 [==============================] - 7s 82ms/step - loss: 0.2896 - accuracy: 0.8650 - val_loss: 0.3479 - val_accuracy: 0.8388
Epoch 41/200
81/81 [==============================] - 7s 81ms/step - loss: nan - accuracy: 0.5360 - val_loss: nan - val_accuracy: 0.4996
Epoch 42/200
81/81 [==============================] - 7s 81ms/step - loss: nan - accuracy: 0.5001 - val_loss: nan - val_accuracy: 0.4996
Epoch 43/200
81/81 [==

In [8]:
probas_lem = modelLEM.predict(np.array(lem_test_features))
y_pred_lem = proba_to_pred(probas_lem)
results_lem = calculate_metrics(lem_test_labels, y_pred_lem)
results_lem

80/80 [==============================] - 1s 15ms/step


{'balanced_accuracy': 0.8408340669014085,
 'f1_score': 0.85183836912996,
 'precision': 0.7975460122699386,
 'recall': 0.9140625}

# Stemmed data

In [9]:
STEMMED_DATA_PATH = os.path.join('..', 'data', 'stemmed')
stem_train_features, stem_val_features, stem_test_features, stem_train_labels, stem_val_labels, stem_test_labels, stem_vocab_size = prepare_data(
    STEMMED_DATA_PATH)

modelSTEM = buildCharGRUModel(stem_vocab_size, embSize=32, inputSize=320)

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 320)]             0         
                                                                 
 embedding_2 (Embedding)     (None, 320, 32)           3360      
                                                                 
 bidirectional_2 (Bidirectio  (None, 1024)             1677312   
 nal)                                                            
                                                                 
 dropout_2 (Dropout)         (None, 1024)              0         
                                                                 
 dense_2 (Dense)             (None, 1)                 1025      
                                                                 
Total params: 1,681,697
Trainable params: 1,681,697
Non-trainable params: 0
_________________________________________________

In [10]:
# Train CNN model.
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
save_best = keras.callbacks.ModelCheckpoint(os.path.join(MODELS_DIR, "stemmed"), monitor='val_loss',
                                            save_best_only=True, restore_best_weights=True)
modelSTEM.fit(np.array(stem_train_features), np.array(stem_train_labels), batch_size=256, epochs=EPOCHS,
              validation_data=(np.array(stem_val_features), np.array(stem_val_labels)),
              callbacks=[early_stopping, save_best])

Epoch 1/200
81/81 [==============================] - ETA: 0s - loss: 0.6645 - accuracy: 0.6129

INFO:tensorflow:Assets written to: ..\models\charCNN_GRU\stemmed\assets


INFO:tensorflow:Assets written to: ..\models\charCNN_GRU\stemmed\assets


81/81 [==============================] - 12s 137ms/step - loss: 0.6645 - accuracy: 0.6129 - val_loss: 0.5912 - val_accuracy: 0.6247
Epoch 2/200
81/81 [==============================] - ETA: 0s - loss: 0.5836 - accuracy: 0.6535

INFO:tensorflow:Assets written to: ..\models\charCNN_GRU\stemmed\assets


...

INFO:tensorflow:Assets written to: ..\models\charCNN_GRU\stemmed\assets


81/81 [==============================] - 11s 138ms/step - loss: 0.3151 - accuracy: 0.8526 - val_loss: 0.3450 - val_accuracy: 0.8367
Epoch 46/200
81/81 [==============================] - 7s 84ms/step - loss: 0.3135 - accuracy: 0.8511 - val_loss: 0.3525 - val_accuracy: 0.8315
Epoch 47/200
81/81 [==============================] - 7s 84ms/step - loss: 0.3096 - accuracy: 0.8542 - val_loss: 0.3886 - val_accuracy: 0.8371
Epoch 48/200
81/81 [==============================] - ETA: 0s - loss: 0.3093 - accuracy: 0.8554

INFO:tensorflow:Assets written to: ..\models\charCNN_GRU\stemmed\assets


INFO:tensorflow:Assets written to: ..\models\charCNN_GRU\stemmed\assets


81/81 [==============================] - 11s 135ms/step - loss: 0.3093 - accuracy: 0.8554 - val_loss: 0.3444 - val_accuracy: 0.8384
Epoch 49/200
81/81 [==============================] - 7s 84ms/step - loss: 0.3028 - accuracy: 0.8572 - val_loss: 0.3519 - val_accuracy: 0.8288
Epoch 50/200
81/81 [==============================] - 7s 84ms/step - loss: 0.3034 - accuracy: 0.8565 - val_loss: 0.3460 - val_accuracy: 0.8328
Epoch 51/200
81/81 [==============================] - 7s 84ms/step - loss: 0.2981 - accuracy: 0.8594 - val_loss: 0.3511 - val_accuracy: 0.8410
Epoch 52/200
81/81 [==============================] - 7s 84ms/step - loss: 0.2976 - accuracy: 0.8604 - val_loss: 0.3503 - val_accuracy: 0.8423
Epoch 53/200
81/81 [==============================] - 7s 84ms/step - loss: 0.2924 - accuracy: 0.8596 - val_loss: 0.3541 - val_accuracy: 0.8349
Epoch 54/200
81/81 [==============================] - 7s 84ms/step - loss: 0.2900 - accuracy: 0.8616 - val_loss: 0.3640 - val_accuracy: 0.8306
Epoch 55/2

In [11]:
probas_stem = modelSTEM.predict(np.array(stem_test_features))
y_pred_stem = proba_to_pred(probas_stem)
results_stem = calculate_metrics(stem_test_labels, y_pred_stem)
results_stem

80/80 [==============================] - 1s 15ms/step


{'balanced_accuracy': 0.8330172877543036,
 'f1_score': 0.8442174388909157,
 'precision': 0.7919233401779603,
 'recall': 0.90390625}

In [12]:
# Create csv with all results

results_all = pd.DataFrame([results, results_lem, results_stem])
results_all['model'] = 'CharCNN+GRU'
results_all['dataset'] = ['simple', 'lemmatized', 'stemmed']

if not os.path.exists('results'):
    os.makedirs('results')
results_all.to_csv('results/char_cnn_gru.csv', index=False)
results_all

,balanced_accuracy,f1_score,precision,recall,model,dataset
0,0.832231,0.844396,0.788084,0.909375,CharCNN+GRU,simple
1,0.840834,0.851838,0.797546,0.914062,CharCNN+GRU,lemmatized
2,0.833017,0.844217,0.791923,0.903906,CharCNN+GRU,stemmed
